In [1]:
#conda list
import pandas as pd
import numpy as np
import time

Prepare a log.txt if not exists

https://www.freecodecamp.org/news/file-handling-in-python/#:~:text=In%20Python%2C%20you%20use%20the,it%20will%20return%20an%20error.

In [2]:
#
f = open("log.txt","a")

In [3]:
import os.path
#https://stackoverflow.com/questions/82831/how-do-i-check-whether-a-file-exists-without-exceptions
if os.path.isfile('q_table_maze.csv'):
    print('Q-Table CSV Exists.')
    time.sleep(1)
    mode=input('1 for loading trained Q-table,2 for starting afresh. Please choose:')
    
    if mode=="1":
        #当后续的训练想要使用之前得到的Q-table时，导出后，传参给RL_brain.py中的__init__ function，最后赋值给self.q_table
        df=pd.read_csv('q_table_maze.csv',index_col=0)
        #df=pd.read_csv('q_table_output.csv') 默认读取时会自动设置新的index，但是这里我们需要的是将第一列(state)设置为index
        #df.columns发现输出时columns的dtype是int，而现在导入变成了object类型
        df.columns=[0,1,2,3]
    elif mode=="2":
        df=pd.DataFrame(columns=[0,1,2,3],dtype=np.float64) #initialize table
        f.write("\n\n-----Start Afresh-----\n")
else:
    df=pd.DataFrame(columns=[0,1,2,3],dtype=np.float64) #initialize table

Q-Table CSV Exists.
1 for loading trained Q-table,2 for starting afresh. Please choose:1


In [4]:
# have a look at current Q-table
df

,0,1,2,3
"[5.0, 5.0, 35.0, 35.0]",0.000000,2.607335e-10,0.000000,0.000000e+00
"[45.0, 5.0, 75.0, 35.0]",0.000000,0.000000e+00,0.000000,1.462637e-13
"[85.0, 5.0, 115.0, 35.0]",0.000000,-1.000000e-02,0.000000,0.000000e+00
"[125.0, 5.0, 155.0, 35.0]",0.000000,0.000000e+00,0.000000,0.000000e+00
"[125.0, 45.0, 155.0, 75.0]",0.000000,0.000000e+00,0.000000,-1.990000e-02
terminal,0.000000,0.000000e+00,0.000000,0.000000e+00
"[5.0, 45.0, 35.0, 75.0]",0.000000,2.912730e-08,0.000000,0.000000e+00
"[5.0, 85.0, 35.0, 115.0]",0.000000,2.340618e-06,-0.010000,0.000000e+00
"[5.0, 125.0, 35.0, 155.0]",0.000000,0.000000e+00,0.000129,4.067620e-07
"[45.0, 45.0, 75.0, 75.0]",0.000000,-1.990000e-02,-0.019900,0.000000e+00


In [5]:
"""
Reinforcement learning maze example.

Red rectangle:          explorer.
Black rectangles:       hells       [reward = -1].
Yellow bin circle:      paradise    [reward = +1].
All other states:       ground      [reward = 0].

This script is the main part which controls the update method of this example.
The RL is in RL_brain.py.

View more on my tutorial page: https://morvanzhou.github.io/tutorials/
"""

from maze_env import Maze
from RL_brain import QLearningTable
import time

epoches=10
def update():
    count_success=0
    count_steps=0
    start=time.time()
    for episode in range(epoches):
        # initial observation
        observation = env.reset()
        
        #step counter
        steps=0

        while True:
            # fresh env
            env.render()

            # RL choose action based on observation
            action = RL.choose_action(str(observation))
        

            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)
            

            # RL learn from this transition
            RL.learn(str(observation), action, reward, str(observation_))
            

            # swap observation
            observation = observation_
            
            steps+=1
            

            # break while loop when end of this episode
            if done:
                end=time.time()
                if reward==1:
                    count_success+=1
                count_steps+=steps
                print('Episode '+str(episode)+', total steps: '+str(steps)+', reward: ', end='')
                print('1') if reward==1 else print ('-1')
                break

    # end of game
    print('game over, AvgSteps= '+str(round(count_steps/epoches))+', success rate= '+str("%.2f%%"%(count_success/epoches*100)))
    # write into log.txt (append)
    f.writelines([time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(start)),"UTC——",
                  time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(end)),"UTC\n","Epoches: ",str(epoches)
                  ,", AvgSteps: ",str(round(count_steps/epoches)),", success rate: ",str("%.2f%%"%(count_success/epoches*100)),"\n"])
    f.close()
    env.destroy()

if __name__ == "__main__":
    env = Maze()
    #RL = QLearningTable(actions=list(range(env.n_actions)))
    RL = QLearningTable(actions=list(range(env.n_actions)),trained_q_table=df)

    env.after(epoches, update)
    env.mainloop()
    print('\r\nQ-table:\n')
    print(RL.q_table)

Episode 0, total steps: 6, reward: 1
Episode 1, total steps: 6, reward: 1
Episode 2, total steps: 6, reward: 1
Episode 3, total steps: 6, reward: 1
Episode 4, total steps: 6, reward: 1
Episode 5, total steps: 6, reward: 1
Episode 6, total steps: 6, reward: 1
Episode 7, total steps: 6, reward: 1
Episode 8, total steps: 7, reward: 1
Episode 9, total steps: 8, reward: 1
game over, AvgSteps= 6, success rate= 100.00%

Q-table:

                                     0             1             2  \
[5.0, 5.0, 35.0, 35.0]        0.000000  2.846866e-08  0.000000e+00   
[45.0, 5.0, 75.0, 35.0]       0.000000  0.000000e+00  0.000000e+00   
[85.0, 5.0, 115.0, 35.0]      0.000000 -1.000000e-02  0.000000e+00   
[125.0, 5.0, 155.0, 35.0]     0.000000  0.000000e+00  0.000000e+00   
[125.0, 45.0, 155.0, 75.0]    0.000000  0.000000e+00  0.000000e+00   
terminal                      0.000000  0.000000e+00  0.000000e+00   
[5.0, 45.0, 35.0, 75.0]       0.000000  1.167261e-06  0.000000e+00   
[5.0, 85.0, 3

In [6]:
RL.q_table

,0,1,2,3
"[5.0, 5.0, 35.0, 35.0]",0.000000,2.846866e-08,0.000000e+00,0.000000e+00
"[45.0, 5.0, 75.0, 35.0]",0.000000,0.000000e+00,0.000000e+00,1.462637e-13
"[85.0, 5.0, 115.0, 35.0]",0.000000,-1.000000e-02,0.000000e+00,0.000000e+00
"[125.0, 5.0, 155.0, 35.0]",0.000000,0.000000e+00,0.000000e+00,0.000000e+00
"[125.0, 45.0, 155.0, 75.0]",0.000000,0.000000e+00,0.000000e+00,-1.990000e-02
terminal,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
"[5.0, 45.0, 35.0, 75.0]",0.000000,1.167261e-06,0.000000e+00,0.000000e+00
"[5.0, 85.0, 35.0, 115.0]",0.000000,3.740842e-05,-1.000000e-02,0.000000e+00
"[5.0, 125.0, 35.0, 155.0]",0.000000,0.000000e+00,9.071520e-04,4.067620e-07
"[45.0, 45.0, 75.0, 75.0]",0.000000,-1.990000e-02,-1.990000e-02,0.000000e+00


In [7]:
type(RL.q_table)

pandas.core.frame.DataFrame

In [8]:
RL.q_table.size

56

In [9]:
df=RL.q_table
df.index

Index(['[5.0, 5.0, 35.0, 35.0]', '[45.0, 5.0, 75.0, 35.0]',
       '[85.0, 5.0, 115.0, 35.0]', '[125.0, 5.0, 155.0, 35.0]',
       '[125.0, 45.0, 155.0, 75.0]', 'terminal', '[5.0, 45.0, 35.0, 75.0]',
       '[5.0, 85.0, 35.0, 115.0]', '[5.0, 125.0, 35.0, 155.0]',
       '[45.0, 45.0, 75.0, 75.0]', '[45.0, 125.0, 75.0, 155.0]',
       '[85.0, 125.0, 115.0, 155.0]', '[125.0, 85.0, 155.0, 115.0]',
       '[125.0, 125.0, 155.0, 155.0]'],
      dtype='object')

In [10]:
df.columns

Index([0, 1, 2, 3], dtype='int64')

In [11]:
# saving the dataframe
df.to_csv('q_table_maze.csv')

In [12]:
# saving the dataframe without header and index
#df.to_csv('q_table_maze.csv', header=False, index=False)